This project will take in a dataset and then predict the loss of the other 

### Imports and CUDA

In [6]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd

In [7]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Loss Ratio is predicted as 

Loss Ratio=bytes_sent/bytes_retrans​

### Preparing the data

I need to normalize the data first, and the continue from there.

In [8]:

# Load dataset manually using NumPy
file_path = "reno1.log.csv"  # Update with actual file path
# Load the dataset using numpy (semicolon separated)
df = pd.read_csv(file_path, delimiter=";")  # skip_header=1 if there’s a header row

# remove last column, some kind of spacing i
# Drop the last column using pandas
df = df.iloc[:, :-1]  # Select all rows, and all columns except the last one

df['loss_ratio'] = df['bytes_retrans'] / df['bytes_sent']
# Identify constant columns
constant_columns = [col for col in df.columns if len(df[col].unique()) == 1]

constant_columns.append("ssthresh")
print("colunms without any variation: ", constant_columns)
# Drop constant columns from the dataset
df = df.drop(columns=constant_columns)

# Normalize using Min-Max scaling: (X - min) / (max - min)
# data_min = df.min(axis=0)
# data_max = df.max(axis=0)
# df_normalized = (df - data_min) / (data_max - data_min)

# # Now df_normalized is your dataset without the constant columns and normalized
# print(df_normalized.iloc[0])
# print(df_normalized.iloc[3000])

# # Normalize using Min-Max scaling: (X - min) / (max - min)
# data_min = np.min(df, axis=0)  # Find min along each column
# data_max = np.max(df, axis=0)  # Find max along each column
# data_normalized = (df - data_min) / (data_max - data_min)


loss_ratio_tensor = torch.tensor(df['loss_ratio'].values, dtype=torch.float32)
# Convert to PyTorch tensor
#Changed df_normalized back to df first.
data_tensor = torch.tensor(df.values, dtype=torch.float32)

colunms without any variation:  ['wscale', 'mss', 'pmtu', 'rcvmss', 'advmss', 'rcv_space', 'rcv_ssthresh', 'ssthresh']


In [9]:
print("Loss Ratio tensor: ", loss_ratio_tensor.shape)
print("Data tensor: ", data_tensor.shape)

Loss Ratio tensor:  torch.Size([3600])
Data tensor:  torch.Size([3600, 12])


### Splitting the data, 80/10/10

In [10]:
train_size = int(len(data_tensor) *0.8)
X_train, X_test = data_tensor[64:train_size], data_tensor[train_size:]
y_train, y_test = loss_ratio_tensor[64:train_size], loss_ratio_tensor[train_size:]

# Print the shapes to verify the split
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64  # You can adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example of accessing a batch of data
for inputs, targets in train_loader:
    print(f'Inputs: {inputs.shape}, Targets: {targets.shape}')
    
    break  # Only print the first batch for verification


torch.Size([2816, 12]) torch.Size([720, 12])
torch.Size([2816]) torch.Size([720])
Inputs: torch.Size([64, 12]), Targets: torch.Size([64])


###LSTM Model used as the NN.

In [11]:
class LSTM_pt(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM_pt, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        # LSTM cell
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers = self.num_layers, batch_first = True)
        
        # Linear layer for final prediction
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, inputs, cell_state=None, hidden_state=None):
        # Forward pass through the LSTM cell
        if hidden_state is None or cell_state is None:
            hidden_state = torch.zeros(1, inputs.size(0), 20).to(device)
            cell_state = torch.zeros(1, inputs.size(0), 20).to(device)
        hidden = (cell_state, hidden_state)
        output, new_memory = self.lstm(inputs, hidden)
        cell_state, hidden_state = new_memory
        output = self.linear(output[:, -1, :])  # Take only the last time step
        return output, cell_state, hidden_state, # Return correct order

### I will implement sliding window next time, but for now, it will only predict the next value.

In [12]:
for inputs,target in train_loader:
    print("input is: ", inputs.shape)
    inputs = inputs.unsqueeze(1).to(device)  # Reshape inputs
    print("input is: ", inputs.shape)

    print("inputs.size(0)", inputs.size(0))
    break


input is:  torch.Size([64, 12])
input is:  torch.Size([64, 1, 12])
inputs.size(0) 64


In [15]:
def train(model, dataloader,num_layers, hidden_size, num_epochs, learning_rate, device, ):
    # Set the loss function and optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        loss=0
        hidden_state, cell_state = None, None   
        for inputs, targets in dataloader: 
            optimizer.zero_grad()
            inputs = inputs.unsqueeze(1).to(device)  # Reshape inputs
            targets = targets.unsqueeze(1).to(device)
            # Initialize hidden state and cell state for each batch

            if hidden_state is not None:
                hidden_state = hidden_state.detach()
            if cell_state is not None:
                cell_state = cell_state.detach()

            # Forward pass
            output, cell_state, hidden_state = model(inputs.to(device),cell_state, hidden_state)
            # Calculate loss
            loss += criterion(output, targets)
        loss /= len(dataloader)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        if epoch %100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss/len(dataloader)}')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss/len(dataloader)}')

In [16]:
# Define the model parameters
# Following the research paper's instructions
input_size = 12
hidden_size = 20
num_layers = 1 # Can be changed to stack multiple LSTM layers!
output_size = 1
dataloader = train_loader
#Create the model
model = LSTM_pt(input_size, hidden_size, num_layers, output_size).to(device)
train(model, dataloader,num_layers, hidden_size, num_epochs = 100, learning_rate = 0.01, device = device)


Epoch 1/100, Loss: 0.002895166864618659
Epoch 100/100, Loss: 3.598399302973121e-07


### Testing the model

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to track loss
total_val_loss = 0
num_batches = 0

# Define the loss function
criterion = torch.nn.MSELoss()

# Initialize hidden state and cell state
hidden_state, cell_state = None, None  

# Disable gradient computation for validation
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        if batch_idx == len(test_loader) - 1:  
            break  # Skip the last batch
        inputs = inputs.unsqueeze(1).to(device)  # Reshape inputs
        targets = targets.unsqueeze(1).to(device)

        # Initialize hidden state and cell state for each batch
        if hidden_state is not None:
            hidden_state = hidden_state.detach()
        if cell_state is not None:
            cell_state = cell_state.detach()

        # Forward pass
        output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)

        # Compute loss
        loss_value = criterion(output, targets)
        total_val_loss += loss_value.item()
        num_batches += 1

# Compute average loss
avg_val_loss = total_val_loss / num_batches

# Print validation results
print(f'Average Validation Loss: {avg_val_loss:.4f}')


TypeError: unsupported operand type(s) for -: 'DataLoader' and 'int'